# Notebook RPP News

### Importación de librerías

In [ ]:
pip install wget

In [ ]:
pip install unicodecsv

In [82]:
from bs4 import BeautifulSoup
import base64
import wget
import requests
import unicodecsv as csv
import datetime
import datetime
import pytz
import dateutil.parser
import pandas as pd
import time

### Creación de arreglo de días

In [78]:
## get days
iterator_day = datetime.date(2019, 9, 2)
last_publication = datetime.date(2019, 9, 3)
array_days = [str(iterator_day)]
while(iterator_day >= last_publication):
    iterator_day = iterator_day - datetime.timedelta(days=1)
    array_days.append(str(iterator_day))

In [79]:
array_days

['2019-09-02']

### Funciones

In [66]:
#Obtener links de archivo por días
def get_news_rpp(days):  
    links = []
    for day in days:           
        url_day = 'https://rpp.pe/archivo/'+day  
        content = requests.get(url_day).content
        soup_article = BeautifulSoup(content,'lxml')
        print("URL Day===>" + str(url_day))
        articles_day = soup_article.findAll(['article',{'class': 'card'}])
        for article in articles_day:            
            if len(article["class"]) == 1:                       
                if article.find('h3') is not None:
                    if article.find('h3').find('a') is not None:
                        tt = article.find('h3').find('a')["href"]
                        links.append(tt)
    return links

In [13]:
#Obtener el título del artículo
def get_title_rpp(soup_article):
    if soup_article.find("h1",{"itemprop":"headline"}) is not None:
        title = soup_article.find("h1",{"itemprop":"headline"}).text
    else:
        title = "None"
    return title

In [16]:
#Obtener la fecha del artículo
def get_date_rpp(soup_article):
    if soup_article.find("time",{"itemprop":"datePublished"}) is not None:
        fecha = soup_article.find("time",{"itemprop":"datePublished"})
        fecha_return = dateutil.parser.parse(fecha["datetime"])      
    else:
        fecha_return = "None"
    return fecha_return
    

In [18]:
#Obtener categoría
def get_cat_rpp(soup_article):
    if soup_article.find("span",{"itemprop":"name"}) is not None:
        categoria = soup_article.find("span",{"itemprop":"name"}).text
    else:
        categoria = "None"
    return categoria

In [20]:
#Obtener autor
def get_author_rpp(soup_article):
    if soup_article.find("div",{"id":"author-name"}) is not None:
        autor = soup_article.find("div",{"id":"author-name"}).text
    else:
        autor = "None"
    return autor

In [22]:
#Obtener resumen
def get_summary_rpp(soup_article):
    if soup_article.find("div",{"class":"sumary"}) is not None:
        resumen = soup_article.find("div",{"class":"sumary"})
        nonBreakSpace = u'\xa0'
        resumen_return = resumen.text.replace(nonBreakSpace, ' ')
    else:
        resumen_return = "None"
    return resumen_return

In [157]:
#Obtener contenido
def get_content_rpp(soup_article):
    if soup_article.find("div",{"id":"article-body"}) is not None:
        contenido_p = soup_article.find("div",{"id":"article-body"}).findAll(["p","h3"])
        content_line = []
        for cont_p in contenido_p:
            if cont_p.parent.has_attr('class'):
                if cont_p.parent.name == "div" and cont_p.parent.attrs['class'][0] == "col-detail":
                    tt = cont_p.text
                    content_line.append(tt+" ")
        
        contenido = ''.join(content_line)
        nonBreakSpace = u'\xa0'
        contenido_return = contenido.replace(nonBreakSpace, ' ')        
    else:
        contenido_return = "None"
    return contenido_return

In [163]:
#Función principal
columns = ["Fecha", "Categoria", "Autor", "Titulo", "Resumen", "Contenido"]
d = []
df_RPP = []
def iterate_articles_rpp(days):
    links = get_news_rpp(days)   
    i = 0
    for link in links:        
        url_article = 'https://rpp.pe' + link
        content_article = requests.get(url_article).content
        soup_article = BeautifulSoup(content_article,'lxml')
        print("URL Article===>" + str(url_article))
        
        date = get_date_rpp(soup_article)     
        category = get_cat_rpp(soup_article)
        author = get_author_rpp(soup_article)
        title = get_title_rpp(soup_article)
        summary = get_summary_rpp(soup_article)
        content = get_content_rpp(soup_article)
        
        d.append((date, category, author, title, summary, content))
        i = i+1  
        print('-------------------- End Article ----------------------')    
    df = pd.DataFrame(d,columns=columns)
    return df

In [164]:
start = time.time()
df_RPP = iterate_articles_rpp(array_days)
end =time.time()
te = end-start
print("Tiempo de ejecución es %f segundos" %(te))

URL Day===>https://rpp.pe/archivo/2019-09-02
URL Article===>https://rpp.pe/mundo/actualidad/huracan-dorian-en-vivo-el-poderoso-ciclon-de-categoria-4-se-dirige-a-territorio-de-estados-unidos-noticia-1217192
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/politica/congreso/presidenta-de-la-comision-de-salud-del-congreso-mas-de-700-neonatos-han-muerto-este-ano-video-noticia-1217562
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/peru/la-libertad/reportan-37-bebes-fallecidos-en-hospital-de-trujillo-tenemos-insuficiente-equipamiento-noticia-1217563
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/famosos/farandula/nicola-porcella-tras-acusaciones-de-agresion-soy-humano-no-soy-perfecto-noticia-1217558
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/futbol/seleccion-peruana/seleccion-peruana-renato-tapia-sobre-los-amistosos-ahora-es-el-momento-

URL Article===>https://rpp.pe/famosos/celebridades/keanu-reeves-cumple-55-anos-su-ascenso-caida-resurgimiento-como-novio-de-internet-fotos-noticia-1217495
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/politica/congreso/ministra-de-educacion-cuestiona-que-comision-del-congreso-busque-investigar-licenciamientos-de-la-sunedu-noticia-1217497
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/campanas/publirreportaje/estas-afiliado-a-una-afp-esto-es-lo-que-debes-saber-de-tu-cuenta-individual-de-capitalizacion-cic-noticia-1216966
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/tecnologia/redes-sociales/porque-todos-lo-pidieron-comercial-de-chupetin-trujillo-en-el-valor-de-la-verdad-se-viraliza-en-redes-noticia-1217501
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/futbol/peruanos-en-el-mundo/fichajes-2019-seleccion-peruana-ivan-bulos-es-el-nu

URL Article===>https://rpp.pe/peru/apurimac/apurimac-un-sismo-de-magnitud-48-remecio-esta-manana-el-distrito-de-chuquibambilla-noticia-1217440
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/economia/economia/un-brindis-en-la-oficina-podria-justificar-un-despido-noticia-1217457
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/musica/nacional/leslie-shaw-video-de-faldita-supero-las-50-millones-de-visitas-y-la-cantante-celebra-los-he-visto-cantar-y-bailarla-noticia-1217444
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/ciencia/espacio/nasa-asi-luce-el-huracan-dorian-desde-el-espacio-noticia-1217445
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/futbol/futbol-mundial/real-madrid-psg-fichajes-2019-keylor-navas-revelo-su-meta-en-el-paris-saint-germain-costa-rica-noticia-1217442
-------------------- End Article ----------------------
URL Art

In [165]:
df_RPP

,Fecha,Categoria,Autor,Titulo,Resumen,Contenido
0,2019-09-02 22:45:06-05:00,Actualidad,Redacción,EN VIVO | Huracán Dorian deja cinco muertos en...,El huracán Dorian se debilitó levemente durant...,El poderoso huracán Dorian dejó al menos cinco...
1,2019-09-02 22:25:46-05:00,Congreso,Redacción,Presidenta de la Comisión de Salud del Congres...,Según la legisladora Rebeca Cruz Tévez (Alianz...,La legisladora Rebeca Cruz Tévez (Alianza para...
2,2019-09-02 22:22:49-05:00,La Libertad,Redacción,Reportan 37 bebés fallecidos en hospital de Tr...,La jefa del área de Neonatalogía del Hospital ...,Treinta y siete bebés prematuros han fallecido...
3,2019-09-02 22:03:45-05:00,Farándula,Redacción,Nicola Porcella tras acusaciones de agresión: ...,El exchico reality continúa publicando mensaje...,Nicola Porcella sorprendió a todos sus seguido...
4,2019-09-02 21:47:31-05:00,Selección Peruana,Redacción,"Renato Tapia sobre los amistosos: ""Ahora es el...",El volante Renato Tapia habló en exclusiva con...,El volante Renato Tapia habló sobre su situaci...
5,2019-09-02 20:57:09-05:00,Panamericanos,Redacción,"¡Campeón! Israel Hilario: ""Mi medalla de oro e...","El para ciclista Israel Hilario, medallista en...",El para deportista Israel Hilario se pronunció...
6,2019-09-02 20:48:55-05:00,Estados Unidos,Redacción,Al menos ocho muertos y 26 desaparecidos tras ...,La mayoría de los tripulantes de la embarcació...,Al menos ocho personas murieron el lunes en el...
7,2019-09-02 20:45:37-05:00,Selección Peruana,Redacción,Selección Peruana : Carlos Zambrano fue el gr...,El defensa de la Selección Peruana solicitó pe...,"De los seis ""extranjeros"" que faltaban unirse ..."
8,2019-09-02 20:12:20-05:00,Fútbol Mundial,Redacción,¡Lo aman! Lluvia de felicitaciones desde Costa...,El portero Keylor Navas dejó Real Madrid tras ...,El fichaje de Keylor Navas por el PSG ha gener...
9,2019-09-02 20:03:12-05:00,Actualidad,Redacción,Caso incubadoras | 63 bebés murieron este año ...,Solo cuatro equipos están operativos. Mauro Sa...,La muerte de niños prematuros por falta de inc...


In [158]:
get_content_rpp(soup_article)

'Nicola Porcella sorprendió a todos sus seguidores a publicar una reflexión en su cuenta de Instagram en medio de las acusaciones de agresión verbal contra su expareja Angie Arizaga. "Me caigo, me levanto; me equivoco, aprendo; me han herido, pero estoy vivo. Soy humano, no soy perfecto, pero estoy agradecido", escribió el exchico reality en la mencionada red social. La publicación ya lleva más de 36 mil likes por parte de sus seguidores, además, acumula miles de comentarios que lo respaldan y le piden que esta vez sí "aprenda de sus errores". Como se recuerda, hace unas semanas, Nicola Porcella fue captado afuera de una discoteca en Miraflores agrediendo verbalmente a su expareja Angie Arizaga. Por este motivo, fue separado de los programas en los que participaba como "Esto es guerra" y "Estás en todas". Además, participó de una nueva edición de "El valor de la verdad", donde reveló su versión de los hechos. MENSAJE PARA SU HIJO No han sido semanas tranquilas para el exintegrante de “